In [1]:
!pip install fastapi uvicorn
!pip install  nest-asyncio pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/bt_project/Pneumonia Detection

/content/drive/.shortcut-targets-by-id/1YmoEWdqPSqxMjsplCqaX8AxWqhP3N5Uh/Pneumonia Detection


# 0. 모델 불러오기

In [4]:
import numpy as np
import cv2
import pickle
from tensorflow.keras.models import load_model
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# 서버 관리용 fastapi 의존 라이브러리
import uvicorn

# fast api 라이브러리
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

import nest_asyncio
from pyngrok import ngrok

In [5]:
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

In [6]:
pwd

'/content/drive/.shortcut-targets-by-id/1YmoEWdqPSqxMjsplCqaX8AxWqhP3N5Uh/Pneumonia Detection'

In [6]:
with open("./model_dict_03.pkl","rb") as fr:
    loaded_dict = pickle.load(fr)

In [7]:
loadedModel = load_model("./model_02.h5")

In [8]:
loaded_dict

{'IMAGE_HEIGHT': 256,
 'IMAGE_WIDTH': 256,
 'LABEL_NAMES': {0: 'NORMAL', 1: 'PNEUMONIA'}}

In [9]:
loadedModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 126, 126, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 conv2d_3 (Conv2D)           (None, 122, 122, 32)      9248      
                                                        

In [10]:
# 이미지 전처리
def ImgPreprocess(file, height, width): # file path가 아닐 수도 있음
  images = []
  image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
  resizedImg = cv2.resize(image, (height, width))
  resizedImg = np.expand_dims(resizedImg, axis = -1)
  image = resizedImg.astype("float32")
  image /=255.0
  images.append(image)

  return np.array(images)

In [12]:
# 이미지 전처리
def ImgPreprocess2(byteImg, height, width): # file path가 아닐 수도 있음
  images = []
  image_data = base64.b64decode(byteImg)
  np_img = np.frombuffer(image_data, np.uint8)
  image = cv2.imdecode(np_img, cv2.IMREAD_COLOR)
  resizedImg = cv2.resize(image, (height, width))
  resizedImg = np.expand_dims(resizedImg, axis = -1)
  image = resizedImg.astype("float32")
  image /=255.0
  images.append(image)

  return np.array(images)

# 디버깅

In [12]:
# file = "./chest_xray/train/PNEUMONIA/person1000_bacteria_2931.jpeg"

In [13]:
# from IPython.display import Image

In [14]:
# Image(file)

In [15]:
# height = loaded_dict['IMAGE_HEIGHT']
# width = loaded_dict['IMAGE_WIDTH']

In [16]:
# ex_img = ImgPreprocess(file, height, width)

In [17]:
# ex_img.shape

In [18]:
# predictions = list(loadedModel.predict(ex_img))[0][0]
# predictions

In [19]:
# binary_predictions = (predictions > 0.8).astype(int)
# binary_predictions

In [20]:
# loaded_dict

In [21]:
# loaded_dict["LABEL_NAMES"][binary_predictions]

In [13]:
def analyze_img(x):
    IMG = x

    # 입력데이터 전처리
    I_height = loaded_dict['IMAGE_HEIGHT']
    I_width = loaded_dict['IMAGE_WIDTH']
    img_after = ImgPreprocess2(IMG, I_height, I_width )

    # 예측
    prediction = list(loadedModel.predict(img_after))[0][0]
    binary_prediction = (prediction > 0.8).astype(int)

    # result: 결과, round_percent: 예측 확률
    result = loaded_dict["LABEL_NAMES"][binary_prediction]
    round_percent = np.round(prediction * 100, 4)

    out = [result, round_percent]

    return result

In [23]:
# analyze_img(file)

NameError: name 'file' is not defined



---



In [14]:
class InDataset(BaseModel):
    image : str

In [15]:
@app.post("/predictdl", status_code=200)
async def analyze(x:InDataset):
    inImg = x.image
    # 화면입력데이터 변수 할당
    predictValue = analyze_img(inImg)
    result = {"prediction": predictValue}
    return result

In [18]:
@app.get("/")
async def root():
    return {"message":"server is running"}

In [17]:
auth_token = "2hRkf178DVW2GeZ80LKi5gn2mNv_asAAzZCc2Zbi1f71TwS6"
ngrok.set_auth_token(auth_token)
ngrokTunnel = ngrok.connect(8000)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [216]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


공용 URL https://684e-34-85-242-241.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [216]


KeyboardInterrupt: 